# データインポート

In [66]:
import numpy as np
import pandas as pd
import os
import sys
from datetime import datetime, timedelta

sys.path.append("../")
from backend.elastic_manager.elastic_manager import ElasticManager

from elasticsearch import Elasticsearch, helpers

ElasticManager.es = Elasticsearch(hosts='http://doc-elastic:9200', http_auth=("elastic", "P@ssw0rd"), timeout=50000)
ElasticManager.es

<Elasticsearch([{'host': 'doc-elastic', 'port': 9200}])>

## 概要確認

In [61]:
# 全ショットデータ
df_angle = pd.read_csv("20210810/0to1K/log0/log000_角度とR2107311654.csv", encoding="shift-jis")
df_angle.head(5)

,DATE,TIME,SHOT数,角度,R半径,板厚SHOT,板厚,板形状SHOT,ピークtoボトム,ピーク距離,ボトム距離,ピークボトム距離
0,2021/07/31,16:54:25,1,15.0283,71.8934,20,1.406,21,5.7352,87.900,194.625,108.975
1,2021/07/31,16:54:27,2,15.5637,74.3223,21,1.466,22,5.8597,73.575,194.625,123.150
2,2021/07/31,16:54:29,3,17.0261,74.7403,22,1.477,23,5.8780,88.875,194.250,108.075
3,2021/07/31,16:54:31,4,18.2962,73.1533,23,1.428,24,5.3836,80.025,194.625,116.325
4,2021/07/31,16:54:33,5,17.0462,71.8284,24,1.435,25,5.1383,87.150,194.625,109.800


In [62]:
# 1ショット分のデータ
df_stroke = pd.read_csv("20210810/0to1K/log1/log000_ストロークと歪210731165425.csv", encoding="shift-jis")
df_stroke.head(10)

,DATE,TIME,INTERVAL,SHOT数,歪量,ストローク
0,2021/07/31,16:54:25,0,9,-7,19.79
1,2021/07/31,16:54:25,99,9,-7,19.79
2,2021/07/31,16:54:25,102,9,-6,19.76
3,2021/07/31,16:54:25,98,9,-6,19.76
4,2021/07/31,16:54:25,107,9,-5,19.75
5,2021/07/31,16:54:25,93,9,-5,19.75
6,2021/07/31,16:54:25,104,9,-6,19.77
7,2021/07/31,16:54:25,96,9,-6,19.77
8,2021/07/31,16:54:25,105,9,4,19.77
9,2021/07/31,16:54:25,95,9,4,19.77


In [63]:
ElasticManager.delete_index("j-20210731165425-data")
ElasticManager.delete_index("j-20210731165425-meta")

ElasticManager.create_index("j-20210731165425-data")
ElasticManager.create_index("j-20210731165425-meta")

[2021/10/08 01:08:06.493] INFO - elastic_manager.py#delete_index:80: j-20210731165425-data deleted.
[2021/10/08 01:08:06.599] INFO - elastic_manager.py#delete_index:80: j-20210731165425-meta deleted.


In [67]:
df_angle["timestamp"] = df_angle["DATE"] + ' ' + df_angle["TIME"]
df_angle["timestamp"] = df_angle["timestamp"].map(lambda x: datetime.strptime(x, "%Y/%m/%d %H:%M:%S") - timedelta(hours=9))
df_angle.drop(columns=["DATE", "TIME"], inplace=True)

In [68]:
dict_angle = df_angle.to_dict(orient="records")
dict_angle

[{'SHOT数': 1,
  '角度': 15.0283,
  'R半径': 71.8934,
  '板厚SHOT': 20,
  '板厚': 1.406,
  '板形状SHOT': 21,
  'ピークtoボトム': 5.7352,
  'ピーク距離': 87.9,
  'ボトム距離': 194.625,
  'ピークボトム距離': 108.975,
  'timestamp': Timestamp('2021-07-31 07:54:25')},
 {'SHOT数': 2,
  '角度': 15.5637,
  'R半径': 74.3223,
  '板厚SHOT': 21,
  '板厚': 1.466,
  '板形状SHOT': 22,
  'ピークtoボトム': 5.8597,
  'ピーク距離': 73.575,
  'ボトム距離': 194.625,
  'ピークボトム距離': 123.15,
  'timestamp': Timestamp('2021-07-31 07:54:27')},
 {'SHOT数': 3,
  '角度': 17.0261,
  'R半径': 74.7403,
  '板厚SHOT': 22,
  '板厚': 1.477,
  '板形状SHOT': 23,
  'ピークtoボトム': 5.878,
  'ピーク距離': 88.875,
  'ボトム距離': 194.25,
  'ピークボトム距離': 108.075,
  'timestamp': Timestamp('2021-07-31 07:54:29')},
 {'SHOT数': 4,
  '角度': 18.2962,
  'R半径': 73.1533,
  '板厚SHOT': 23,
  '板厚': 1.428,
  '板形状SHOT': 24,
  'ピークtoボトム': 5.3836,
  'ピーク距離': 80.025,
  'ボトム距離': 194.625,
  'ピークボトム距離': 116.325,
  'timestamp': Timestamp('2021-07-31 07:54:31')},
 {'SHOT数': 5,
  '角度': 17.0462,
  'R半径': 71.8284,
  '板厚SHOT': 24,
  '板厚': 1.435,
 

In [69]:
actions = (
    {"_index": "j-20210731165425-meta", "_source": x} for x in dict_angle
)
helpers.bulk(ElasticManager.es, actions, chunk_size=5000, stats_only=True, raise_on_error=False)

(1000, 0)

In [70]:
import glob

files = glob.glob("20210810/0to1K/log1/*")
files.sort()

df_stroke = pd.read_csv(files[0], encoding="shift-jis")

for file in files[1:]:
    df = pd.read_csv(file, encoding="shift-jis")
    df_stroke = pd.concat([df_stroke, df])

df_stroke.head(5)

,DATE,TIME,INTERVAL,SHOT数,歪量,ストローク
0,2021/07/31,16:54:25,0,9,-7,19.79
1,2021/07/31,16:54:25,99,9,-7,19.79
2,2021/07/31,16:54:25,102,9,-6,19.76
3,2021/07/31,16:54:25,98,9,-6,19.76
4,2021/07/31,16:54:25,107,9,-5,19.75


In [71]:
df_stroke["timestamp"] = df_stroke["DATE"] + ' ' + df_stroke["TIME"]
df_stroke["timestamp"] = df_stroke["timestamp"].map(lambda x: datetime.strptime(x, "%Y/%m/%d %H:%M:%S") - timedelta(hours=9))
df_stroke.drop(columns=["DATE", "TIME"], inplace=True)

In [88]:
df_stroke

,INTERVAL,SHOT数,歪量,ストローク,timestamp
0,0,9,-7,19.79,2021-07-31 07:54:25
1,99,9,-7,19.79,2021-07-31 07:54:25
2,102,9,-6,19.76,2021-07-31 07:54:25
3,98,9,-6,19.76,2021-07-31 07:54:25
4,107,9,-5,19.75,2021-07-31 07:54:25
...,...,...,...,...,...
2939,99,1008,19,20.03,2021-07-31 08:31:21
2940,100,1008,19,20.03,2021-07-31 08:31:21
2941,99,1008,9,20.03,2021-07-31 08:31:21
2942,101,1008,9,20.01,2021-07-31 08:31:21


In [89]:
dict_stroke = df_stroke.to_dict(orient="records")
# dict_stroke

In [90]:
actions = (
    {"_index": "j-20210731165425-data", "_source": x} for x in dict_stroke
)
helpers.bulk(ElasticManager.es, actions, chunk_size=5000, stats_only=True, raise_on_error=False)

(2942132, 0)

## 角度外れ値除去

In [79]:
df_angle[df_angle["角度"] < 0]

,SHOT数,角度,R半径,板厚SHOT,板厚,板形状SHOT,ピークtoボトム,ピーク距離,ボトム距離,ピークボトム距離,timestamp
914,915,-100000.0,73.1096,934,1.512,935,4.2556,69.6,194.25,124.65,2021-07-31 08:28:23


In [80]:
df_angle_without_outlire = df_angle.drop(914)

In [83]:
df_angle_without_outlire[df_angle_without_outlire["角度"] < 0]

,SHOT数,角度,R半径,板厚SHOT,板厚,板形状SHOT,ピークtoボトム,ピーク距離,ボトム距離,ピークボトム距離,timestamp


### 再インポート

In [85]:
ElasticManager.delete_index("j-20210731165425-meta")
ElasticManager.create_index("j-20210731165425-meta")

[2021/10/08 01:26:36.565] INFO - elastic_manager.py#delete_index:80: j-20210731165425-meta deleted.
[2021/10/08 01:26:36.995] INFO - elastic_manager.py#create_index:193: create index 'j-20210731165425-meta' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'j-20210731165425-meta'}


True

In [84]:
dict_angle_without_outlire = df_angle_without_outlire.to_dict(orient="records")

In [87]:
actions = (
    {"_index": "j-20210731165425-meta", "_source": x} for x in dict_angle_without_outlire
)
helpers.bulk(ElasticManager.es, actions, chunk_size=5000, stats_only=True, raise_on_error=False)

(999, 0)